In [1]:
import os
import cv2
import random
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd 

In [2]:
def writeFile(pathList, fileName):
    # print(content)
    file = open(fileName, "a")

    for path in pathList:
        file.write(path)
        file.write("\n")

    file.close()

def getMultipliedDataPath():
    # get augmentedDataPath
    with open("augmentedDataPath.txt", "r") as file:
        pathLines = file.read().splitlines() 

    # get path multiplier
    with open("dataPathMultiplier.txt", "r") as file:
        multiplierList = file.read().splitlines() 

    newDataPath = []

    # loop into each path
    for path in pathLines:
        # get data key for data
        dataKey = path.split('/')[2][4:]

        # loop into multiplier to create complete data path to assign data key
        for multiplier in multiplierList:
            completePath = path + multiplier
            newDataPath.append([dataKey, completePath])
            
    return newDataPath

def getWheatData():
    # get wheat data
    with open("2024allLabelData.csv", "r") as file:
        wheatDataList = file.read().splitlines()[1:]

    # change wheat data to dict, using dataKey as key
    wheatDataDict = {}
    for wheatData in wheatDataList:
        wheatData = wheatData.split(',')
        wheatDataDict[wheatData[0]] = wheatData[1:]
    return (wheatDataDict)

def linkData(newDataPath, wheatDataDict):
    ''' 
    link data path list 
    and data dict
    using dataKey, if not match change key to 999999999999_xx
    '''
    completeLabelDataList = []

    # loop into dataPath
    for dataPathKey in newDataPath:
        #separate path and key
        dataKey = dataPathKey[0]
        dataPath = dataPathKey[1]

        # check if key match, if not, assign 999999999999_xx as key
        if (dataKey not in [*wheatDataDict]):
            dataKey = "999999999999_" + dataKey.split('_')[-1]
        
        # get wheat data using key
        thisPathDictData = wheatDataDict[dataKey]

        # combine path, key , [wheat data] to 1 list
        completeLabelData = [dataPath, dataKey]
        for eachData in thisPathDictData:
            completeLabelData.append(eachData)

        completeLabelDataList.append((','.join(completeLabelData)))

    return completeLabelDataList

In [ ]:

newDataPath = getMultipliedDataPath()
wheatDataDict = getWheatData()
completeLabelDataList = linkData(newDataPath,wheatDataDict)
# dataPathPandas = pd.DataFrame(newDataPath)
# dataPathPandas.columns = ['DataKey', 'DataPath']
# wheatData = pd.read_csv("2024allLabelData.csv")
# writeFile(completeLabelDataList, "/Volumes/HD-PCFSU3-A/ice-wheat/data/dataForProcess/mainData/completeLabelData.txt")